In [12]:
%pip install langchain chromadb sentence-transformers pypdf openai numpy pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
%pip install langchain-community

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
%pip install tf-keras

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
%pip install "pypdf[full]"

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
from langchain.document_loaders import PyPDFLoader
import os
from pypdf import PdfReader
import sys

# Use the BCSC 1807 Syllabus PDF
pdf_path = r"C:\OMEGA\Python\GenAi\Assignment\Assignment - 3\RAG\BCSC1807 Syllabus.pdf"

# Check if file exists
print(f"Checking if file exists at: {pdf_path}")
if os.path.exists(pdf_path):
    print("File exists. Loading PDF...")
    try:
        # First try direct PDF reading
        reader = PdfReader(pdf_path)
        print(f"Number of pages (direct reading): {len(reader.pages)}")
        text = reader.pages[0].extract_text()
        if text.strip():
            print("\nSample content from first page (direct reading):")
            print(text[:500])
        else:
            print("\nWarning: No text could be extracted from the first page")
        
        # Then try with LangChain loader
        print("\nNow trying with LangChain loader:")
        loader = PyPDFLoader(pdf_path)
        pages = loader.load()
        print(f"Total pages loaded: {len(pages)}")
        
        if not any(page.page_content.strip() for page in pages):
            print("Warning: No content could be extracted from the PDF. The file might be scanned or protected.")
    except Exception as e:
        print(f"Error processing PDF: {str(e)}")
        sys.exit(1)
else:
    print(f"File not found at: {pdf_path}")
    print("Please make sure the PDF file exists at the specified location.")

Checking if file exists at: C:\OMEGA\Python\GenAi\Assignment\Assignment - 3\RAG\BCSC1807 Syllabus.pdf
File exists. Loading PDF...
Number of pages (direct reading): 1

Sample content from first page (direct reading):
                                                                          
 
DEPARTMENT OF COMPUTER ENGINEERING & APPLICATIONS, Institute of Engineering & Technology 
Course Curriculum (w.e.f. Session 2023-24) B. Tech. Computer Science & Engineering 
BCSC1807: DESIGN & ANALYSIS OF ALGORITHMS LAB Objective: The objective of this course is that students will understand and implement simple data structures, able demonstrate different sorting and searching techniques. and will be familiar with graph

Now trying with LangChain loader:
Total pages loaded: 1


In [18]:
print("Sample content from first page:")
if len(pages) > 0:
    print(pages[0].page_content[:500])

Sample content from first page:
DEPARTMENT OF COMPUTER ENGINEERING & APPLICATIONS, Institute of Engineering & Technology 
Course Curriculum (w.e.f. Session 2023-24) B. Tech. Computer Science & Engineering 
BCSC1807: DESIGN & ANALYSIS OF ALGORITHMS LAB Objective: The objective of this course is that students will understand and implement simple data structures, able demonstrate different sorting and searching techniques. and will be familiar with graphs and their applications.   Credits:01     L-T-P-J:0-0-2-0 Module No. Content


In [19]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Increased chunk size
    chunk_overlap=200,  # Increased overlap
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)
docs = splitter.split_documents(pages)

print(f"Total chunks created: {len(docs)}")

Total chunks created: 4


In [20]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

In [21]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=docs, embedding=embedding_model, persist_directory="./chroma_db")
vectorstore.persist()
print("Embedding stored in Chroma.")

Embedding stored in Chroma.


c:\Users\anike\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
C:\Users\anike\AppData\Local\Temp\ipykernel_18496\2329283953.py:4: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [22]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [23]:
from langchain.chains import RetrievalQA
from langchain.llms.base import LLM
from typing import Optional, List

class DummyLLM(LLM):
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        return "This is a simulated LLM response to the prompt: " + prompt

    @property
    def _llm_type(self) -> str:
        return "dummy-llm"

qa_chain = RetrievalQA.from_chain_type(llm=DummyLLM(), retriever=retriever)

In [24]:
query = "What is the document about?"
response = qa_chain.run(query)
print("Answer:", response)

Answer: This is a simulated LLM response to the prompt: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

DEPARTMENT OF COMPUTER ENGINEERING & APPLICATIONS, Institute of Engineering & Technology 
Course Curriculum (w.e.f. Session 2023-24) B. Tech. Computer Science & Engineering

BCSC1807: DESIGN & ANALYSIS OF ALGORITHMS LAB Objective: The objective of this course is that students will understand and implement simple data structures, able demonstrate different sorting and searching techniques. and will be familiar with graphs and their applications.   Credits:01     L-T-P-J:0-0-2-0 Module No. Content  Teaching Hours        I& II          Implementation of sorting algorithms: o Insertion Sort o Bubble Sort o Selection Sort  Divide and conquer approach: o Quick Sort o Merge Sort  Implementation of sorting algorithms: o Heap Sort  o Counting Sort   Implementation of Searchi

C:\Users\anike\AppData\Local\Temp\ipykernel_18496\3251645052.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)
c:\Users\anike\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [25]:
questions = [
    "What is the main topic of the document?",
    "Which methods or models are used in the study?",
    "What are the key results or findings?",
    "Is there any future work or limitation mentioned?",
    "What is the purpose or objective of this document?"
]

In [26]:
def ask_with_retriever(qa_chain, retriever, query):
    relevant_docs = retriever.get_relevant_documents(query)
    response = qa_chain.run(query)

    print(f"\n Question: {query}")
    print("\n Retrieved Chunks:")
    for i, doc in enumerate(relevant_docs):
        print(f"\nChunk {i+1}:\n{doc.page_content[:300]}...\n")

    print(" RAG Answer:\n", response)
    return response

In [27]:
def ask_without_retriever(llm, query):
    # Simulate raw LLM answer using full doc context (concat first few chunks)
    context = "\n".join([doc.page_content for doc in docs[:5]])
    prompt = f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"
    answer = llm(prompt)

    print(" Raw LLM Answer:\n", answer)
    return answer

In [28]:
llm=DummyLLM()

In [29]:
for q in questions:
    ask_with_retriever(qa_chain, retriever, q)
    ask_without_retriever(llm, q)
    print("-" * 100)


 Question: What is the main topic of the document?

 Retrieved Chunks:

Chunk 1:
DEPARTMENT OF COMPUTER ENGINEERING & APPLICATIONS, Institute of Engineering & Technology 
Course Curriculum (w.e.f. Session 2023-24) B. Tech. Computer Science & Engineering...


Chunk 2:
BCSC1807: DESIGN & ANALYSIS OF ALGORITHMS LAB Objective: The objective of this course is that students will understand and implement simple data structures, able demonstrate different sorting and searching techniques. and will be familiar with graphs and their applications.   Credits:01     L-T-P-J:...


Chunk 3:
data structures for a given problem • CO3: Implement the algorithms based on Greedy approach and Dynamic Programming.    Mapping of Course Outcomes (COs) with Program Outcomes (POs) and Program Specific Outcomes (PSOs):  COs POs/PSOs CO1 PO1,PO2,PO4/PSO1,PSO2,PSO4 CO2 PO1,PO3,PO4/PSO1,PSO2,PSO3 CO3 ...

 RAG Answer:
 This is a simulated LLM response to the prompt: Use the following pieces of context to answer the

C:\Users\anike\AppData\Local\Temp\ipykernel_18496\2149761887.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = retriever.get_relevant_documents(query)
c:\Users\anike\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
c:\Users\anike\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
C:\Users\anike\AppData\Local\Temp\ipykernel_18496\2441574792.py:5: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langcha